In [1]:
import socketserver
import http.server
import urllib
import base64
import pickle
import http
import socket
import select
import requests
import threading
import os

from requests.packages.urllib3.exceptions import InsecureRequestWarning
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

In [2]:
class HTTPServer(socketserver.ThreadingTCPServer):
    
    def __init__(self, addr, request_handler):
        self._thread = None
        self.is_stopped = False
        super().__init__(addr, request_handler)
    
    # from http.server.HTTPServer
    def server_bind(self):
        """Override server_bind to store the server name."""
        socketserver.TCPServer.server_bind(self)
        host, port = self.socket.getsockname()[:2]
        self.server_name = socket.getfqdn(host)
        self.server_port = port
        
    def serve_forever(self, interval=0.05):
        # I attempted to speed up response time by overriding the default 
        # serve_forever with a shorter interval, but it doesn't seem to have worked.
        # oh well. 
        target = super().serve_forever
        self._thread = threading.Thread(target=target, args=(interval,), daemon=True)
        self._thread.start()
        
    def stop(self):
        if self._thread is not None:
            self.shutdown()
            self._thread.join()
            self._thread = None
            self.socket.close()
            self.is_stopped = True
    

In [22]:
def parse_params(params):
    out = {}
    for k, v in params.items():
        if len(v) > 1:
            raise ValueError("Too many arguments for %r: %r" % (k, ", ".join(v)))
        out[k] = v[0]
    return out

def parse_qs(qs):
    return parse_params(urllib.parse.parse_qs(qs))

class HelloProxyError(Exception):
    def __init__(self, code, msg):
        self.response = ProxyResponse(code, msg, "text/plain", {'Connection': 'Close'})
        super().__init__(msg)
        
def get_file_content(fp):
    with open(fp, 'rb') as f:
        return f.read()
        
def content_type_for_file(file):
    ext = os.path.splitext(file)[1]
    if ext == '.json':
        return 'Application/json'
    elif ext == '.css':
        return 'text/css'
    elif ext == '.png':
        return 'image/png'
    elif ext == '.js':
        return 'application/x-javascript'
    elif ext == '.gif':
        return 'image/gif'
    elif ext == '.ico':
        return 'image/icon'
    elif ext == '.html':
        return 'text/html'
    else:
        raise HelloProxyError(http.HTTPStatus.BAD_REQUEST, "Unrecognized MIME type")
        
def byte_encode(msg):
    if isinstance(msg, str):
        return msg.encode('ascii')
    elif isinstance(msg, bytes):
        return msg
    else:
        raise HelloProxyError(http.HTTPStatus.INTERNAL_SERVER_ERROR, "Error encoding response data")
        
class ProxyResponse():
    def __init__(self, code, msg, headers=None):
        self.code = code
        self.msg = byte_encode(msg)
        self.headers = headers or {}
        
def proxy_response_for_file(fullpath):
    msg = get_file_content(fullpath)
    
    headers = {
        'Content-Type': content_type_for_file(fullpath),
        'Content-Length': str(len(msg)),
        'Connection': 'Keep-Alive'
    }
    
    return ProxyResponse(200, msg, headers)
        
class HelloProxyHandler(http.server.SimpleHTTPRequestHandler):
    
    def __init__(self, *args, **kw):
        self.sess = requests.Session()
        super().__init__(*args, **kw)
    
    def handle(self):
        self.close_connection = False
        while not self.close_connection:
            r, w, l = select.select([self.request], [], [], 60*10)
            if self.server.is_stopped:
                break
            if r:
                self.handle_one_request()
            else:
                break
    
    def do_GET(self):
        res = urllib.parse.urlparse(self.path) 
        kw = parse_qs(res.query)
        
        try:
            rsp = self._handle_get_internal(res, kw)
        except HelloProxyError as e:
            rsp = e.response
        except Exception as e:
            msg = "???" + e.args[0]
            headers = {
                'Connection': 'Close',
                'Content-Type': 'text/plain',
                'Content-Length': str(len(msg))
            }
            rsp = ProxyResponse(http.HTTPStatus.INTERNAL_SERVER_ERROR, msg, headers)
                  
        if rsp.headers['Connection'] == 'Close':
            self.close_connection = True
            
        self.send_response(rsp.code)
        for k, v in rsp.headers.items():
            self.send_header(k, v)
        self.end_headers()
        self.wfile.write(rsp.msg)
        self.wfile.flush()
        
    def _handle_get_internal(self, res, kw):

        # reject paths that try to navigate out of 
        # the local directory
        if ".." in res.path:
            raise HelloProxyError(http.HTTPStatus.BAD_REQUEST, "Path requests cannot contain access local paths via '..'.")
        
        if res.path.startswith('/webservice'):
            _, ws, handler = res.path.split('/',2)
            if handler == 'interface/':
                return self._handle_server_call(res, kw)
            else:
                raise HelloProxyError(http.HTTPStatus.BAD_REQUEST, "Unsupported service path: '%s'"%handler)
        fullpath = os.path.join(self.server.basepath, res.path.lstrip("\\/"))
        if os.path.exists(fullpath):
            return proxy_response_for_file(fullpath)
        else:
            raise HelloProxyError(http.HTTPStatus.NOT_FOUND, "Failed to find file '%s'"%res.path)
        
        raise HelloProxyError(http.HTTPStatus.INTERNAL_SERVER_ERROR, "Unreachable code path")
        
    def _handle_server_call(self, res, kw):
        call = kw.pop('call', None)
        if call is None:
            self._server_call_bad_params("no call provided with query string")
        url = self.server.proxy_url + self.path
        
        hfwd = dict(self.headers)
        hfwd['Host'] = '%s:%s' % self.server.remoteaddr
        fwd = self.sess.get(url, headers=hfwd, verify=False)
        
        msg = fwd.content
        fheaders = fwd.headers
        
        headers = {}
        if 'Set-Cookie' in fheaders:
            headers['Set-Cookie'] = fheaders['Set-Cookie']
        if 'Content-Type' in fheaders:
            headers['Content-Type'] = fheaders['Content-Type']
        headers['Connection'] = 'Keep-Alive'
        headers['Content-Length'] = str(len(msg))
        
        rsp = ProxyResponse(fwd.status_code, msg, fwd.headers) 
        global prsp, fheaders
        if call == 'login':
            prsp = rsp
            fheaders = headers
        return rsp
            
    def _server_call_bad_params(self, call, **kw):
        if kw:
            kws = ", ".join("%s=%s" for i in kw.items())
            kws = ": '%s'" % kws
        else:
            kws = ""
        if call is None:
            call = "No call argument provided"
        else:
            call = '%s'%call
            
        msg = "Bad params for server call: %s%s" % (call, kws)
        raise HelloProxyError(http.HTTPStatus.BAD_REQUEST, msg)
        
class HelloProxyServer(HTTPServer):
    def __init__(self, localaddr, remoteaddr, basepath):
        self.remoteaddr = remoteaddr
        self.basepath = basepath
        self.proxy_url = "https://%s"%remoteaddr[0]
        super().__init__(localaddr, HelloProxyHandler)
        
    def __del__(self):
        self.socket.close()

127.0.0.1 - - [27/Mar/2019 14:02:14] "GET /webservice/interface/?&call=getMainValues&json=true&_=1553720145596 HTTP/1.1" 200 -
<ipython-input-22-2d2b30d95d6e>:152: SyntaxWarning: name 'fheaders' is assigned to before global declaration
  global prsp, fheaders
127.0.0.1 - - [27/Mar/2019 14:02:15] "GET /webservice/interface/?&call=getDORAValues&_=1553720145597 HTTP/1.1" 200 -


In [24]:
path = "C:\\Users\\natha\\Documents\\PBS\\hello38"
serv = HelloProxyServer(('localhost', 12345), ('192.168.1.9', 443), path)
serv.serve_forever()

127.0.0.1 - - [27/Mar/2019 14:02:21] "GET /webservice/interface/?&call=getMainValues&json=true&_=1553720145599 HTTP/1.1" 200 -
127.0.0.1 - - [27/Mar/2019 14:02:23] "GET /webservice/interface/?&call=getUsers&_=1553720145600 HTTP/1.1" 200 -
127.0.0.1 - - [27/Mar/2019 14:02:25] "GET /webservice/interface/?&call=getUnAckCount&_=1553720145601 HTTP/1.1" 200 -
127.0.0.1 - - [27/Mar/2019 14:02:25] "GET /webservice/interface/?&call=getDORAValues&_=1553720145602 HTTP/1.1" 200 -
127.0.0.1 - - [27/Mar/2019 14:02:36] "GET /webservice/interface/?&call=getLight&dataType=json&skipValidate=true&json=true&_=1553720145603 HTTP/1.1" 200 -
127.0.0.1 - - [27/Mar/2019 14:02:47] "GET /webservice/interface/?&call=getMainValues&json=true&_=1553720145604 HTTP/1.1" 200 -
127.0.0.1 - - [27/Mar/2019 14:02:50] "GET /webservice/interface/?&call=getUsers&_=1553720145605 HTTP/1.1" 200 -
127.0.0.1 - - [27/Mar/2019 14:02:52] "GET /webservice/interface/?&call=getUnAckCount&_=1553720145606 HTTP/1.1" 200 -
127.0.0.1 - - [27

127.0.0.1 - - [27/Mar/2019 14:07:52] "GET /webservice/interface/?&call=getUnAckCount&_=1553720145665 HTTP/1.1" 200 -
127.0.0.1 - - [27/Mar/2019 14:07:55] "GET /webservice/interface/?&call=getDORAValues&_=1553720145666 HTTP/1.1" 200 -
127.0.0.1 - - [27/Mar/2019 14:07:59] "GET /webservice/interface/?&call=getMainValues&json=true&_=1553720145667 HTTP/1.1" 200 -
127.0.0.1 - - [27/Mar/2019 14:08:02] "GET /webservice/interface/?&call=getLight&dataType=json&skipValidate=true&json=true&_=1553720145668 HTTP/1.1" 200 -
127.0.0.1 - - [27/Mar/2019 14:08:15] "GET /webservice/interface/?&call=getUsers&_=1553720145669 HTTP/1.1" 200 -
127.0.0.1 - - [27/Mar/2019 14:08:17] "GET /webservice/interface/?&call=getUnAckCount&_=1553720145670 HTTP/1.1" 200 -
127.0.0.1 - - [27/Mar/2019 14:08:20] "GET /webservice/interface/?&call=getDORAValues&_=1553720145671 HTTP/1.1" 200 -
127.0.0.1 - - [27/Mar/2019 14:08:25] "GET /webservice/interface/?&call=getMainValues&json=true&_=1553720145672 HTTP/1.1" 200 -
127.0.0.1 - 

127.0.0.1 - - [27/Mar/2019 14:13:56] "GET /webservice/interface/?&call=getDORAValues&_=1553720145732 HTTP/1.1" 200 -
127.0.0.1 - - [27/Mar/2019 14:13:56] "GET /webservice/interface/?&call=getUnAckCount&_=1553720145731 HTTP/1.1" 200 -
127.0.0.1 - - [27/Mar/2019 14:14:03] "GET /webservice/interface/?&call=getMainValues&json=true&_=1553720145733 HTTP/1.1" 200 -
127.0.0.1 - - [27/Mar/2019 14:14:26] "GET /webservice/interface/?&call=getDORAValues&_=1553720145735 HTTP/1.1" 200 -
127.0.0.1 - - [27/Mar/2019 14:14:26] "GET /webservice/interface/?&call=getUnAckCount&_=1553720145734 HTTP/1.1" 200 -
127.0.0.1 - - [27/Mar/2019 14:14:29] "GET /webservice/interface/?&call=getMainValues&json=true&_=1553720145736 HTTP/1.1" 200 -
127.0.0.1 - - [27/Mar/2019 14:14:36] "GET /webservice/interface/?&call=getLight&dataType=json&skipValidate=true&json=true&_=1553720145737 HTTP/1.1" 200 -
127.0.0.1 - - [27/Mar/2019 14:14:55] "GET /webservice/interface/?&call=getMainValues&json=true&_=1553720145738 HTTP/1.1" 200

127.0.0.1 - - [27/Mar/2019 14:23:26] "GET /webservice/interface/?&call=getUnAckCount&_=1553720145798 HTTP/1.1" 200 -
127.0.0.1 - - [27/Mar/2019 14:23:26] "GET /webservice/interface/?&call=getDORAValues&_=1553720145799 HTTP/1.1" 200 -
127.0.0.1 - - [27/Mar/2019 14:23:35] "GET /webservice/interface/?&call=getMainValues&json=true&_=1553720145800 HTTP/1.1" 200 -
127.0.0.1 - - [27/Mar/2019 14:23:56] "GET /webservice/interface/?&call=getLight&dataType=json&skipValidate=true&json=true&_=1553720145801 HTTP/1.1" 200 -
127.0.0.1 - - [27/Mar/2019 14:23:56] "GET /webservice/interface/?&call=getDORAValues&_=1553720145803 HTTP/1.1" 200 -
127.0.0.1 - - [27/Mar/2019 14:23:56] "GET /webservice/interface/?&call=getUnAckCount&_=1553720145802 HTTP/1.1" 200 -
127.0.0.1 - - [27/Mar/2019 14:24:01] "GET /webservice/interface/?&call=getMainValues&json=true&_=1553720145804 HTTP/1.1" 200 -
127.0.0.1 - - [27/Mar/2019 14:24:26] "GET /webservice/interface/?&call=getUnAckCount&_=1553720145805 HTTP/1.1" 200 -
127.0.0

127.0.0.1 - - [27/Mar/2019 14:32:56] "GET /webservice/interface/?&call=getUnAckCount&_=1553720145865 HTTP/1.1" 200 -
127.0.0.1 - - [27/Mar/2019 14:32:56] "GET /webservice/interface/?&call=getDORAValues&_=1553720145866 HTTP/1.1" 200 -
127.0.0.1 - - [27/Mar/2019 14:33:07] "GET /webservice/interface/?&call=getMainValues&json=true&_=1553720145867 HTTP/1.1" 200 -
127.0.0.1 - - [27/Mar/2019 14:33:16] "GET /webservice/interface/?&call=getLight&dataType=json&skipValidate=true&json=true&_=1553720145868 HTTP/1.1" 200 -
127.0.0.1 - - [27/Mar/2019 14:33:26] "GET /webservice/interface/?&call=getDORAValues&_=1553720145870 HTTP/1.1" 200 -
127.0.0.1 - - [27/Mar/2019 14:33:26] "GET /webservice/interface/?&call=getUnAckCount&_=1553720145869 HTTP/1.1" 200 -
127.0.0.1 - - [27/Mar/2019 14:33:33] "GET /webservice/interface/?&call=getMainValues&json=true&_=1553720145871 HTTP/1.1" 200 -
127.0.0.1 - - [27/Mar/2019 14:33:56] "GET /webservice/interface/?&call=getDORAValues&_=1553720145873 HTTP/1.1" 200 -
127.0.0

...

127.0.0.1 - - [27/Mar/2019 16:47:53] "GET /webservice/interface/?&call=getMainValues&json=true&_=1553720146817 HTTP/1.1" 200 -
127.0.0.1 - - [27/Mar/2019 16:47:56] "GET /webservice/interface/?&call=getLight&dataType=json&skipValidate=true&json=true&_=1553720146818 HTTP/1.1" 200 -
127.0.0.1 - - [27/Mar/2019 16:47:56] "GET /webservice/interface/?&call=getUnAckCount&_=1553720146819 HTTP/1.1" 200 -
127.0.0.1 - - [27/Mar/2019 16:47:56] "GET /webservice/interface/?&call=getDORAValues&_=1553720146820 HTTP/1.1" 200 -
127.0.0.1 - - [27/Mar/2019 16:48:19] "GET /webservice/interface/?&call=getMainValues&json=true&_=1553720146821 HTTP/1.1" 200 -
127.0.0.1 - - [27/Mar/2019 16:48:26] "GET /webservice/interface/?&call=getUnAckCount&_=1553720146822 HTTP/1.1" 200 -
127.0.0.1 - - [27/Mar/2019 16:48:26] "GET /webservice/interface/?&call=getDORAValues&_=1553720146823 HTTP/1.1" 200 -
127.0.0.1 - - [27/Mar/2019 16:48:45] "GET /webservice/interface/?&call=getMainValues&json=true&_=1553720146824 HTTP/1.1" 200

127.0.0.1 - - [27/Mar/2019 16:57:16] "GET /webservice/interface/?&call=getLight&dataType=json&skipValidate=true&json=true&_=1553720146884 HTTP/1.1" 200 -
127.0.0.1 - - [27/Mar/2019 16:57:25] "GET /webservice/interface/?&call=getMainValues&json=true&_=1553720146885 HTTP/1.1" 200 -
127.0.0.1 - - [27/Mar/2019 16:57:26] "GET /webservice/interface/?&call=getDORAValues&_=1553720146887 HTTP/1.1" 200 -
127.0.0.1 - - [27/Mar/2019 16:57:26] "GET /webservice/interface/?&call=getUnAckCount&_=1553720146886 HTTP/1.1" 200 -
127.0.0.1 - - [27/Mar/2019 16:57:51] "GET /webservice/interface/?&call=getMainValues&json=true&_=1553720146888 HTTP/1.1" 200 -
127.0.0.1 - - [27/Mar/2019 16:57:55] "GET /webservice/interface/?&call=getUnAckCount&_=1553720146889 HTTP/1.1" 200 -
127.0.0.1 - - [27/Mar/2019 16:57:56] "GET /webservice/interface/?&call=getDORAValues&_=1553720146890 HTTP/1.1" 200 -
127.0.0.1 - - [27/Mar/2019 16:58:17] "GET /webservice/interface/?&call=getMainValues&json=true&_=1553720146891 HTTP/1.1" 200

127.0.0.1 - - [27/Mar/2019 17:06:36] "GET /webservice/interface/?&call=getLight&dataType=json&skipValidate=true&json=true&_=1553720146951 HTTP/1.1" 200 -
127.0.0.1 - - [27/Mar/2019 17:06:56] "GET /webservice/interface/?&call=getDORAValues&_=1553720146953 HTTP/1.1" 200 -
127.0.0.1 - - [27/Mar/2019 17:06:56] "GET /webservice/interface/?&call=getUnAckCount&_=1553720146952 HTTP/1.1" 200 -
127.0.0.1 - - [27/Mar/2019 17:06:57] "GET /webservice/interface/?&call=getMainValues&json=true&_=1553720146954 HTTP/1.1" 200 -
127.0.0.1 - - [27/Mar/2019 17:07:23] "GET /webservice/interface/?&call=getMainValues&json=true&_=1553720146955 HTTP/1.1" 200 -
127.0.0.1 - - [27/Mar/2019 17:07:26] "GET /webservice/interface/?&call=getUnAckCount&_=1553720146956 HTTP/1.1" 200 -
127.0.0.1 - - [27/Mar/2019 17:07:26] "GET /webservice/interface/?&call=getDORAValues&_=1553720146957 HTTP/1.1" 200 -
127.0.0.1 - - [27/Mar/2019 17:07:49] "GET /webservice/interface/?&call=getMainValues&json=true&_=1553720146958 HTTP/1.1" 200

127.0.0.1 - - [27/Mar/2019 17:16:05] "GET /webservice/interface/?&call=getMainValues&json=true&_=1553720147018 HTTP/1.1" 200 -
127.0.0.1 - - [27/Mar/2019 17:16:26] "GET /webservice/interface/?&call=getUnAckCount&_=1553720147019 HTTP/1.1" 200 -
127.0.0.1 - - [27/Mar/2019 17:16:26] "GET /webservice/interface/?&call=getDORAValues&_=1553720147020 HTTP/1.1" 200 -
127.0.0.1 - - [27/Mar/2019 17:16:29] "GET /webservice/interface/?&call=getMainValues&json=true&_=1553720147021 HTTP/1.1" 200 -
127.0.0.1 - - [27/Mar/2019 17:16:55] "GET /webservice/interface/?&call=getMainValues&json=true&_=1553720147022 HTTP/1.1" 200 -
127.0.0.1 - - [27/Mar/2019 17:16:55] "GET /webservice/interface/?&call=getUnAckCount&_=1553720147023 HTTP/1.1" 200 -
127.0.0.1 - - [27/Mar/2019 17:16:56] "GET /webservice/interface/?&call=getDORAValues&_=1553720147024 HTTP/1.1" 200 -
127.0.0.1 - - [27/Mar/2019 17:17:16] "GET /webservice/interface/?&call=getLight&dataType=json&skipValidate=true&json=true&_=1553720147025 HTTP/1.1" 200

127.0.0.1 - - [27/Mar/2019 17:25:35] "GET /webservice/interface/?&call=getMainValues&json=true&_=1553720147085 HTTP/1.1" 200 -
127.0.0.1 - - [27/Mar/2019 17:25:56] "GET /webservice/interface/?&call=getUnAckCount&_=1553720147086 HTTP/1.1" 200 -
127.0.0.1 - - [27/Mar/2019 17:25:56] "GET /webservice/interface/?&call=getDORAValues&_=1553720147087 HTTP/1.1" 200 -
127.0.0.1 - - [27/Mar/2019 17:26:01] "GET /webservice/interface/?&call=getMainValues&json=true&_=1553720147088 HTTP/1.1" 200 -
127.0.0.1 - - [27/Mar/2019 17:26:26] "GET /webservice/interface/?&call=getDORAValues&_=1553720147090 HTTP/1.1" 200 -
127.0.0.1 - - [27/Mar/2019 17:26:26] "GET /webservice/interface/?&call=getUnAckCount&_=1553720147089 HTTP/1.1" 200 -
127.0.0.1 - - [27/Mar/2019 17:26:26] "GET /webservice/interface/?&call=getMainValues&json=true&_=1553720147091 HTTP/1.1" 200 -
127.0.0.1 - - [27/Mar/2019 17:26:36] "GET /webservice/interface/?&call=getLight&dataType=json&skipValidate=true&json=true&_=1553720147092 HTTP/1.1" 200

In [23]:
serv.stop()
del serv

In [18]:
fheaders

{'Date': 'Wed, 27 Mar 2019 20:55:49 GMT', 'Set-Cookie': 'a8mheiXN#wu7jcnBecknUw!!=aswI4H09dnhXAsV972bilw!!; Path=/; HttpOnly; Secure', 'Cache-Control': 'no-cache, no-cache="Set-Cookie"', 'Connection': 'Keep-Alive', 'Keep-Alive': 'timeout=60, max=193', 'Server': 'Embedthis-http', 'Content-Type': 'application/json', 'Content-Length': '28'}

In [21]:
len(prsp.msg)

28

127.0.0.1 - - [27/Mar/2019 13:58:14] "GET /webservice/interface/?&call=getMainValues&json=true&_=1553720145433 HTTP/1.1" 200 -
127.0.0.1 - - [27/Mar/2019 13:58:15] "GET /webservice/interface/?&call=getDORAValues&_=1553720145434 HTTP/1.1" 200 -
127.0.0.1 - - [27/Mar/2019 13:58:16] "GET /webservice/interface/?&call=getMainValues&json=true&_=1553720145435 HTTP/1.1" 200 -
127.0.0.1 - - [27/Mar/2019 13:58:18] "GET /webservice/interface/?&call=getMainValues&json=true&_=1553720145436 HTTP/1.1" 200 -
127.0.0.1 - - [27/Mar/2019 13:58:20] "GET /webservice/interface/?&call=getMainValues&json=true&_=1553720145437 HTTP/1.1" 200 -
127.0.0.1 - - [27/Mar/2019 13:58:22] "GET /webservice/interface/?&call=getMainValues&json=true&_=1553720145438 HTTP/1.1" 200 -
127.0.0.1 - - [27/Mar/2019 13:58:24] "GET /webservice/interface/?&call=getMainValues&json=true&_=1553720145439 HTTP/1.1" 200 -
127.0.0.1 - - [27/Mar/2019 13:58:25] "GET /webservice/interface/?&call=getUnAckCount&_=1553720145440 HTTP/1.1" 200 -
127.

127.0.0.1 - - [27/Mar/2019 13:59:52] "GET /webservice/interface/?&call=getMainValues&json=true&_=1553720145499 HTTP/1.1" 200 -
127.0.0.1 - - [27/Mar/2019 13:59:54] "GET /webservice/interface/?&call=getMainValues&json=true&_=1553720145500 HTTP/1.1" 200 -
127.0.0.1 - - [27/Mar/2019 13:59:55] "GET /webservice/interface/?&call=getUnAckCount&_=1553720145502 HTTP/1.1" 200 -
127.0.0.1 - - [27/Mar/2019 13:59:55] "GET /webservice/interface/?&call=getLight&dataType=json&skipValidate=true&json=true&_=1553720145501 HTTP/1.1" 200 -
127.0.0.1 - - [27/Mar/2019 13:59:56] "GET /webservice/interface/?&call=getDORAValues&_=1553720145503 HTTP/1.1" 200 -
127.0.0.1 - - [27/Mar/2019 13:59:56] "GET /webservice/interface/?&call=getMainValues&json=true&_=1553720145504 HTTP/1.1" 200 -
127.0.0.1 - - [27/Mar/2019 13:59:58] "GET /webservice/interface/?&call=getMainValues&json=true&_=1553720145505 HTTP/1.1" 200 -
127.0.0.1 - - [27/Mar/2019 14:00:00] "GET /webservice/interface/?&call=getMainValues&json=true&_=1553720

127.0.0.1 - - [27/Mar/2019 14:01:26] "GET /webservice/interface/?&call=getMainValues&json=true&_=1553720145565 HTTP/1.1" 200 -
127.0.0.1 - - [27/Mar/2019 14:01:28] "GET /webservice/interface/?&call=getMainValues&json=true&_=1553720145566 HTTP/1.1" 200 -
127.0.0.1 - - [27/Mar/2019 14:01:30] "GET /webservice/interface/?&call=getMainValues&json=true&_=1553720145567 HTTP/1.1" 200 -
127.0.0.1 - - [27/Mar/2019 14:01:32] "GET /webservice/interface/?&call=getMainValues&json=true&_=1553720145568 HTTP/1.1" 200 -
127.0.0.1 - - [27/Mar/2019 14:01:34] "GET /webservice/interface/?&call=getMainValues&json=true&_=1553720145569 HTTP/1.1" 200 -
127.0.0.1 - - [27/Mar/2019 14:01:35] "GET /webservice/interface/?&call=getDORAValues&_=1553720145570 HTTP/1.1" 200 -
127.0.0.1 - - [27/Mar/2019 14:01:36] "GET /webservice/interface/?&call=getMainValues&json=true&_=1553720145571 HTTP/1.1" 200 -
127.0.0.1 - - [27/Mar/2019 14:01:38] "GET /webservice/interface/?&call=getMainValues&json=true&_=1553720145572 HTTP/1.1" 